# IWRC Seed Fund Return on Investment Analysis

## Purpose
This analysis demonstrates the **return on investment (ROI)** of IWRC's seed funding program by quantifying:
- Total funding allocated to researchers
- Follow-on grants and awards secured using IWRC seed funding as preliminary data
- Students trained through funded projects
- Geographic and institutional reach across Illinois

**Time Periods Analyzed:**
- **10-Year Period:** 2015-2024
- **5-Year Period:** 2020-2024

---

## Setup: Import Libraries and Configure Settings

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import warnings
warnings.filterwarnings('ignore')

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

# Configure visualization settings for professional output
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 150  # High DPI for one-pager graphics
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Professional color palette (blue-orange for contrast)
COLORS = {
    'primary': '#1f77b4',    # Blue
    'secondary': '#ff7f0e',  # Orange
    'success': '#2ca02c',    # Green
    'accent': '#d62728',     # Red
    'purple': '#9467bd',
    'brown': '#8c564b',
    'pink': '#e377c2'
}

print('✓ All libraries imported successfully!')
print('✓ Professional styling configured for publication-ready outputs')

In [ ]:
# Load the consolidated Excel file
file_path = 'IWRC Seed Fund Tracking.xlsx'
df = pd.read_excel(file_path, sheet_name='Project Overview')

print(f'✓ Data loaded successfully!')
print(f'  Total rows: {len(df):,}')
print(f'  Total columns: {len(df.columns)}')
print(f'\nDataset covers fiscal years 2016-2024 with {df["Project ID "].nunique()} unique projects')

In [ ]:
# Create simplified column name mappings for easier reference
col_map = {
    'Project ID ': 'project_id',
    'Award Type': 'award_type',
    'Project Title': 'project_title',
    'Project PI': 'pi_name',
    'Academic Institution of PI': 'institution',
    'Award Amount Allocated ($) this must be filled in for all lines': 'award_amount',
    'Number of PhD Students Supported by WRRA $': 'phd_students',
    'Number of MS Students Supported by WRRA $': 'ms_students',
    'Number of Undergraduate Students Supported by WRRA $': 'undergrad_students',
    'Number of Post Docs Supported by WRRA $': 'postdoc_students',
    "Award, Achievement, or Grant\n (This may include awards and achievements for projects from the previous year to this 5-year cycle, so long as they were not already included in last year's report)": 'awards_grants',
    "Description of Award, Achievement, or Grant\n (This may include awards and achievements for projects from the previous year to this 5-year cycle, so long as they were not already included in last year's report)": 'award_description',
    'Monetary Benefit of Award or Achievement (if applicable; use NA if not applicable)': 'monetary_benefit',
    'WRRI Science Priority that Best Aligns with this Project': 'science_priority',
    'Keyword (Primary)': 'keyword_primary'
}

# Rename columns for easier access
df_work = df.rename(columns=col_map)

# Clean student columns - convert non-numeric values to NaN
student_cols = ['phd_students', 'ms_students', 'undergrad_students', 'postdoc_students']
for col in student_cols:
    df_work[col] = pd.to_numeric(df_work[col], errors='coerce')

print('✓ Column names simplified for analysis')
print('✓ Student columns cleaned (non-numeric values converted to NaN)')

---
## Step 2: Extract Year from Project ID and Create Time Period Filters

**How we identify project years:**  
Project IDs contain year information in various formats (e.g., "2015-001", "2020-IL103AIS", "FY16-XXX"). We extract the 4-digit year to categorize projects into our analysis periods.

In [ ]:
def extract_year_from_project_id(project_id):
    """
    Extract year from Project ID using multiple pattern matching strategies.
    
    Handles formats like:
    - "2015-001" -> 2015
    - "2020-IL103AIS" -> 2020
    - "FY16-XXX" -> 2016
    """
    if pd.isna(project_id):
        return None
    
    project_id_str = str(project_id).strip()
    
    # Try to find 4-digit year (20XX or 19XX)
    year_match = re.search(r'(20\d{2}|19\d{2})', project_id_str)
    if year_match:
        return int(year_match.group(1))
    
    # Try to find FY format (FY16 -> 2016)
    fy_match = re.search(r'FY(\d{2})', project_id_str, re.IGNORECASE)
    if fy_match:
        fy_year = int(fy_match.group(1))
        # Convert 2-digit year to 4-digit (assuming 20XX)
        return 2000 + fy_year if fy_year < 100 else fy_year
    
    return None

# Apply year extraction
df_work['project_year'] = df_work['project_id'].apply(extract_year_from_project_id)

# Display year extraction results
print('Year Extraction Summary:')
print(f'  Projects with valid year: {df_work["project_year"].notna().sum():,}')
print(f'  Projects without year: {df_work["project_year"].isna().sum():,}')
print(f'\nYear Range: {df_work["project_year"].min():.0f} - {df_work["project_year"].max():.0f}')
print(f'\nProjects by Year:')
print(df_work['project_year'].value_counts().sort_index().tail(10))

In [ ]:
# Create filtered datasets for our two analysis periods
df_10yr = df_work[df_work['project_year'].between(2015, 2024, inclusive='both')].copy()
df_5yr = df_work[df_work['project_year'].between(2020, 2024, inclusive='both')].copy()

print('✓ Time period filters created:')
print(f'  10-Year Period (2015-2024): {len(df_10yr):,} rows')
print(f'  5-Year Period (2020-2024): {len(df_5yr):,} rows')

---
## Step 3: Analysis #1 - IWRC Investment (Total Funding Allocated)

**What this shows:**  
The total dollar amount IWRC invested in seed funding to support water research projects across Illinois.

In [ ]:
# Calculate total IWRC investment for each period
investment_10yr = df_10yr['award_amount'].sum()
investment_5yr = df_5yr['award_amount'].sum()

# Count UNIQUE projects by Project ID (not rows)
num_projects_10yr = df_10yr['project_id'].nunique()
num_projects_5yr = df_5yr['project_id'].nunique()

# Create summary table
investment_summary = pd.DataFrame({
    'Time Period': ['10-Year (2015-2024)', '5-Year (2020-2024)'],
    'Total IWRC Investment': [investment_10yr, investment_5yr],
    'Number of Projects': [num_projects_10yr, num_projects_5yr]
})

# Format currency
investment_summary['Total IWRC Investment ($)'] = investment_summary['Total IWRC Investment'].apply(lambda x: f'${x:,.2f}')
investment_summary = investment_summary[['Time Period', 'Total IWRC Investment ($)', 'Number of Projects']]

print('\n' + '='*70)
print('IWRC SEED FUNDING INVESTMENT SUMMARY')
print('='*70)
print(investment_summary.to_string(index=False))
print('='*70)

In [ ]:
# Visualization: IWRC Investment Comparison
fig, ax = plt.subplots(figsize=(10, 6))

periods = ['10-Year\n(2015-2024)', '5-Year\n(2020-2024)']
investments = [investment_10yr, investment_5yr]

bars = ax.barh(periods, investments, color=[COLORS['primary'], COLORS['secondary']], height=0.6)

# Add value labels on bars
for i, (bar, value) in enumerate(zip(bars, investments)):
    ax.text(value + max(investments)*0.02, i, f'${value:,.0f}', 
            va='center', fontsize=12, fontweight='bold')

ax.set_xlabel('Total Investment ($)', fontsize=12, fontweight='bold')
ax.set_title('IWRC Seed Funding Investment by Time Period', fontsize=14, fontweight='bold', pad=20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('iwrc_investment_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Chart saved as: iwrc_investment_comparison.png')

In [ ]:
# Create award_category column by categorizing the awards_grants field
def categorize_award(award_text):
    """
    Categorize awards into Grant, Award, or Achievement based on text.
    """
    if pd.isna(award_text):
        return None
    
    award_str = str(award_text).lower()
    
    if 'grant' in award_str:
        return 'Grant'
    elif 'award' in award_str:
        return 'Award'
    elif 'achievement' in award_str:
        return 'Achievement'
    else:
        return 'Other'

# Apply categorization to both datasets
df_10yr['award_category'] = df_10yr['awards_grants'].apply(categorize_award)
df_5yr['award_category'] = df_5yr['awards_grants'].apply(categorize_award)

# Count awards by category
def count_awards_by_category(df):
    """Count non-null awards by category."""
    df_with_awards = df[df['award_category'].notna()].copy()
    if len(df_with_awards) == 0:
        return pd.Series(dtype=int)
    return df_with_awards['award_category'].value_counts()

awards_10yr = count_awards_by_category(df_10yr)
awards_5yr = count_awards_by_category(df_5yr)

print('\n' + '='*70)
print('AWARDS/GRANTS CATEGORIZATION')
print('='*70)
print('\n10-Year Period (2015-2024):')
print(awards_10yr)
print(f'\n  TOTAL AWARDS/GRANTS: {awards_10yr.sum()}')
print('\n5-Year Period (2020-2024):')
print(awards_5yr)
print(f'\n  TOTAL AWARDS/GRANTS: {awards_5yr.sum()}')
print('='*70)

---
## Step 4: Analysis #2 - Follow-on Funding (Awards, Achievements, and Grants)

**What this shows:**  
Researchers use IWRC seed funding to generate preliminary data, which they then leverage to secure larger grants from federal agencies and other sources. This analysis counts how many grants/awards researchers obtained and their total monetary value.

**How we identify grants:**  
We search the "Award, Achievement, or Grant" column for keywords like "grant", "award", and "achievement" (case-insensitive).

In [ ]:
def clean_monetary_value(value):
    """
    Clean and convert monetary values to float with comprehensive extraction.
    Handles: $X,XXX, NA, embedded amounts, text descriptions, etc.
    """
    if pd.isna(value):
        return 0.0

    value_str = str(value).strip().upper()

    # Handle NA or N/A
    if value_str in ['NA', 'N/A', 'NONE', '']:
        return 0.0

    # Try to extract ALL dollar amounts from text and sum them
    # Pattern: $X,XXX.XX or $XXXX or XXXX (with optional commas and decimals)
    dollar_pattern = r'\$?\s*([\d,]+(?:\.\d{2})?)'
    matches = re.findall(dollar_pattern, value_str)

    if matches:
        total = 0.0
        for match in matches:
            try:
                # Remove commas and convert to float
                amount = float(match.replace(',', ''))
                total += amount
            except (ValueError, TypeError):
                continue
        return total if total > 0 else 0.0

    return 0.0

def extract_grant_amount_comprehensive(row):
    """
    Extract grant monetary value from multiple possible columns.
    Handles data entry inconsistencies where amounts appear in different columns.

    Priority order:
    1. monetary_benefit column
    2. award_description column (for swapped data)
    3. Extract from awards_grants text (e.g., "$250,000" in description)
    """
    # Priority 1: Check monetary_benefit column
    if pd.notna(row['monetary_benefit']):
        amount = clean_monetary_value(row['monetary_benefit'])
        if amount > 0:
            return amount

    # Priority 2: Check award_description column (swapped data)
    if pd.notna(row['award_description']):
        amount = clean_monetary_value(row['award_description'])
        if amount > 0:
            return amount

    # Priority 3: Extract from awards_grants text
    if pd.notna(row['awards_grants']):
        amount = clean_monetary_value(row['awards_grants'])
        if amount > 0:
            return amount

    return 0.0

# Apply comprehensive extraction to both datasets
df_10yr['monetary_benefit_clean'] = df_10yr.apply(extract_grant_amount_comprehensive, axis=1)
df_5yr['monetary_benefit_clean'] = df_5yr.apply(extract_grant_amount_comprehensive, axis=1)

# Calculate total monetary benefit by award category
def calculate_monetary_by_category(df):
    # Only include rows with valid award categories
    df_with_awards = df[df['award_category'].notna()].copy()

    if len(df_with_awards) == 0:
        return pd.DataFrame(columns=['Count', 'Total Value'])

    monetary_summary = df_with_awards.groupby('award_category')['monetary_benefit_clean'].agg([
        ('Count', 'count'),
        ('Total Value', 'sum')
    ]).round(2)

    return monetary_summary

monetary_10yr = calculate_monetary_by_category(df_10yr)
monetary_5yr = calculate_monetary_by_category(df_5yr)

# Add data quality report
total_with_amounts_10yr = (df_10yr['monetary_benefit_clean'] > 0).sum()
total_with_amounts_5yr = (df_5yr['monetary_benefit_clean'] > 0).sum()

print('\n' + '='*70)
print('FOLLOW-ON FUNDING: MONETARY VALUE BY TYPE')
print('='*70)
print('\n10-Year Period (2015-2024):')
print(monetary_10yr)
print(f'\n  TOTAL FOLLOW-ON FUNDING: ${monetary_10yr["Total Value"].sum():,.2f}')
print(f'  (Based on {total_with_amounts_10yr} entries with monetary values)')
print('\n5-Year Period (2020-2024):')
print(monetary_5yr)
print(f'\n  TOTAL FOLLOW-ON FUNDING: ${monetary_5yr["Total Value"].sum():,.2f}')
print(f'  (Based on {total_with_amounts_5yr} entries with monetary values)')
print('\n' + '='*70)
print('\nNOTE: Some grant/award amounts may not be reported in the dataset.')
print('These figures represent confirmed follow-on funding with documented values.')
print('='*70)

In [ ]:
# Visualization: Count of Awards/Grants by Type
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# 10-Year Period
if not awards_10yr.empty:
    awards_10yr.plot(kind='bar', ax=ax1, color=[COLORS['primary'], COLORS['secondary'], COLORS['success']])
    ax1.set_title('10-Year Period (2015-2024)', fontsize=13, fontweight='bold')
    ax1.set_xlabel('Type', fontsize=11)
    ax1.set_ylabel('Count', fontsize=11)
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
    
    # Add value labels on bars
    for container in ax1.containers:
        ax1.bar_label(container, fontsize=10, fontweight='bold')

# 5-Year Period
if not awards_5yr.empty:
    awards_5yr.plot(kind='bar', ax=ax2, color=[COLORS['primary'], COLORS['secondary'], COLORS['success']])
    ax2.set_title('5-Year Period (2020-2024)', fontsize=13, fontweight='bold')
    ax2.set_xlabel('Type', fontsize=11)
    ax2.set_ylabel('Count', fontsize=11)
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
    
    # Add value labels on bars
    for container in ax2.containers:
        ax2.bar_label(container, fontsize=10, fontweight='bold')

fig.suptitle('Follow-on Funding: Awards, Achievements, and Grants Secured', 
             fontsize=14, fontweight='bold', y=1.02)

for ax in [ax1, ax2]:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('followon_funding_count.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Chart saved as: followon_funding_count.png')

---
## Step 5: Analysis #3 - Return on Investment (ROI) Calculation

**What this shows:**  
The ROI multiplier demonstrates how effectively IWRC seed funding leverages additional research dollars. For every $1 IWRC invests, researchers secure $X in follow-on funding.

**Formula:**  
ROI Multiplier = Total Follow-on Funding ÷ Total IWRC Investment

In [ ]:
# Calculate ROI for each period
followon_10yr = monetary_10yr['Total Value'].sum()
followon_5yr = monetary_5yr['Total Value'].sum()

roi_10yr = followon_10yr / investment_10yr if investment_10yr > 0 else 0
roi_5yr = followon_5yr / investment_5yr if investment_5yr > 0 else 0

# Create ROI summary table
roi_summary = pd.DataFrame({
    'Time Period': ['10-Year (2015-2024)', '5-Year (2020-2024)'],
    'IWRC Investment': [f'${investment_10yr:,.2f}', f'${investment_5yr:,.2f}'],
    'Follow-on Funding': [f'${followon_10yr:,.2f}', f'${followon_5yr:,.2f}'],
    'ROI Multiplier': [f'{roi_10yr:.2f}x', f'{roi_5yr:.2f}x']
})

print('\n' + '='*80)
print('RETURN ON INVESTMENT (ROI) SUMMARY')
print('='*80)
print(roi_summary.to_string(index=False))
print('='*80)
print(f'\nINTERPRETATION:')
print(f'  10-Year: For every $1 invested, researchers secured ${roi_10yr:.2f} in additional funding')
print(f'  5-Year:  For every $1 invested, researchers secured ${roi_5yr:.2f} in additional funding')
print('='*80)

In [ ]:
# Visualization: ROI Comparison
fig, ax = plt.subplots(figsize=(12, 7))

periods = ['10-Year\n(2015-2024)', '5-Year\n(2020-2024)']
x = np.arange(len(periods))
width = 0.35

# Create grouped bars
bars1 = ax.bar(x - width/2, [investment_10yr, investment_5yr], width, 
               label='IWRC Investment', color=COLORS['primary'])
bars2 = ax.bar(x + width/2, [followon_10yr, followon_5yr], width, 
               label='Follow-on Funding Secured', color=COLORS['success'])

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'${height:,.0f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

# Add ROI multiplier annotations
for i, (period, roi) in enumerate(zip(periods, [roi_10yr, roi_5yr])):
    ax.text(i, max(investment_10yr, investment_5yr, followon_10yr, followon_5yr) * 1.15,
            f'ROI: {roi:.2f}x',
            ha='center', fontsize=12, fontweight='bold', 
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))

ax.set_ylabel('Funding Amount ($)', fontsize=12, fontweight='bold')
ax.set_title('IWRC Seed Funding Return on Investment', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(periods, fontsize=11)
ax.legend(fontsize=11, loc='upper left')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('roi_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Chart saved as: roi_comparison.png')

---
## Step 6: Analysis #4 - Students Trained

**What this shows:**  
The number of students (PhD, Master's, undergraduate, and post-doctoral researchers) who received training and support through IWRC-funded projects. These students gain valuable research experience in water resources science.

In [ ]:
# Calculate student totals for each period
def calculate_student_totals(df):
    student_cols = ['phd_students', 'ms_students', 'undergrad_students', 'postdoc_students']
    
    totals = {}
    for col in student_cols:
        totals[col] = df[col].sum()
    
    totals['total'] = sum(totals.values())
    return totals

students_10yr = calculate_student_totals(df_10yr)
students_5yr = calculate_student_totals(df_5yr)

# Create summary table
student_summary = pd.DataFrame({
    'Student Type': ['PhD', 'Master\'s', 'Undergraduate', 'Post-Doctoral', 'TOTAL'],
    '10-Year (2015-2024)': [
        int(students_10yr['phd_students']),
        int(students_10yr['ms_students']),
        int(students_10yr['undergrad_students']),
        int(students_10yr['postdoc_students']),
        int(students_10yr['total'])
    ],
    '5-Year (2020-2024)': [
        int(students_5yr['phd_students']),
        int(students_5yr['ms_students']),
        int(students_5yr['undergrad_students']),
        int(students_5yr['postdoc_students']),
        int(students_5yr['total'])
    ]
})

print('\n' + '='*70)
print('STUDENTS TRAINED THROUGH IWRC SEED FUNDING')
print('='*70)
print(student_summary.to_string(index=False))
print('='*70)

In [ ]:
# Visualization: Students Trained - Stacked Bar Chart
fig, ax = plt.subplots(figsize=(10, 6))

categories = ['PhD', "Master's", 'Undergraduate', 'Post-Doctoral']
data_10yr = [
    students_10yr['phd_students'],
    students_10yr['ms_students'],
    students_10yr['undergrad_students'],
    students_10yr['postdoc_students']
]
data_5yr = [
    students_5yr['phd_students'],
    students_5yr['ms_students'],
    students_5yr['undergrad_students'],
    students_5yr['postdoc_students']
]

x = np.arange(len(categories))
width = 0.35

colors_list = [COLORS['primary'], COLORS['secondary'], COLORS['success'], COLORS['purple']]

bars1 = ax.bar(x - width/2, data_10yr, width, label='10-Year (2015-2024)', color=COLORS['primary'])
bars2 = ax.bar(x + width/2, data_5yr, width, label='5-Year (2020-2024)', color=COLORS['secondary'])

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')

ax.set_ylabel('Number of Students', fontsize=12, fontweight='bold')
ax.set_title('Students Trained Through IWRC Seed Funding', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=11)
ax.legend(fontsize=11)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('students_trained.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Chart saved as: students_trained.png')

In [ ]:
# Visualization: Student Distribution - Pie Charts
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

colors_pie = [COLORS['primary'], COLORS['secondary'], COLORS['success'], COLORS['purple']]

# 10-Year Period Pie Chart
if students_10yr['total'] > 0:
    ax1.pie(data_10yr, labels=categories, autopct='%1.1f%%', startangle=90, 
            colors=colors_pie, textprops={'fontsize': 11})
    ax1.set_title('10-Year Period (2015-2024)\nTotal Students: {:,.0f}'.format(students_10yr['total']), 
                  fontsize=13, fontweight='bold')

# 5-Year Period Pie Chart
if students_5yr['total'] > 0:
    ax2.pie(data_5yr, labels=categories, autopct='%1.1f%%', startangle=90, 
            colors=colors_pie, textprops={'fontsize': 11})
    ax2.set_title('5-Year Period (2020-2024)\nTotal Students: {:,.0f}'.format(students_5yr['total']), 
                  fontsize=13, fontweight='bold')

fig.suptitle('Distribution of Students Trained by Type', fontsize=14, fontweight='bold', y=1.02)

plt.tight_layout()
plt.savefig('student_distribution_pie.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Chart saved as: student_distribution_pie.png')

---
## Step 7: Analysis #5 - Institutional Reach and Geographic Diversity

**What this shows:**  
IWRC serves the entire state of Illinois, not just the University of Illinois. This analysis demonstrates the geographic reach and institutional diversity of funded projects.

In [ ]:
# Calculate institutional diversity
def analyze_institutions(df, period_name):
    # Get unique institutions (excluding NaN)
    institutions = df['institution'].dropna().unique()
    n_institutions = len(institutions)
    
    # Count projects by institution
    inst_counts = df['institution'].value_counts()
    
    print(f'\n{period_name}:')
    print(f'  Unique Institutions: {n_institutions}')
    print(f'\n  Top Institutions by Project Count:')
    print(inst_counts.head(10).to_string())
    
    return inst_counts

print('\n' + '='*70)
print('INSTITUTIONAL REACH - SERVING ALL OF ILLINOIS')
print('='*70)

inst_10yr = analyze_institutions(df_10yr, '10-Year Period (2015-2024)')
inst_5yr = analyze_institutions(df_5yr, '5-Year Period (2020-2024)')

print('='*70)

---
## Executive Summary: Key Metrics Dashboard

This summary provides the most important metrics at a glance for inclusion in fact sheets and stakeholder presentations.

In [ ]:
# Create comprehensive executive summary
executive_summary = pd.DataFrame({
    'Metric': [
        'Total IWRC Investment',
        'Total Follow-on Funding Secured',
        'Return on Investment (ROI)',
        'Number of Grants Awarded',
        'Total Students Trained',
        'PhD Students',
        'Master\'s Students',
        'Undergraduate Students',
        'Post-Doctoral Researchers',
        'Number of Projects',
        'Institutions Served'
    ],
    '10-Year (2015-2024)': [
        f'${investment_10yr:,.2f}',
        f'${followon_10yr:,.2f}',
        f'{roi_10yr:.2f}x',
        int(awards_10yr.get('Grant', 0)),
        int(students_10yr['total']),
        int(students_10yr['phd_students']),
        int(students_10yr['ms_students']),
        int(students_10yr['undergrad_students']),
        int(students_10yr['postdoc_students']),
        num_projects_10yr,
        df_10yr['institution'].nunique()
    ],
    '5-Year (2020-2024)': [
        f'${investment_5yr:,.2f}',
        f'${followon_5yr:,.2f}',
        f'{roi_5yr:.2f}x',
        int(awards_5yr.get('Grant', 0)),
        int(students_5yr['total']),
        int(students_5yr['phd_students']),
        int(students_5yr['ms_students']),
        int(students_5yr['undergrad_students']),
        int(students_5yr['postdoc_students']),
        num_projects_5yr,
        df_5yr['institution'].nunique()
    ]
})

print('\n' + '='*80)
print('EXECUTIVE SUMMARY: IWRC SEED FUND ROI ANALYSIS')
print('='*80)
print(executive_summary.to_string(index=False))
print('='*80)

print('\n✓ KEY TAKEAWAYS:')
print(f'  • IWRC serves {df_10yr["institution"].nunique()} institutions across Illinois (10-year period)')
print(f'  • For every $1 invested, IWRC generates ${roi_10yr:.2f} in follow-on funding (10-year)')
print(f'  • {int(students_10yr["total"])} students trained over 10 years')
print(f'  • Researchers secured {int(awards_10yr.get("Grant", 0))} grants using IWRC seed funding')
print('='*80)

In [ ]:
# Export executive summary to Excel for easy sharing
with pd.ExcelWriter('IWRC_ROI_Analysis_Summary.xlsx', engine='openpyxl') as writer:
    executive_summary.to_excel(writer, sheet_name='Executive Summary', index=False)
    investment_summary.to_excel(writer, sheet_name='Investment', index=False)
    roi_summary.to_excel(writer, sheet_name='ROI Analysis', index=False)
    student_summary.to_excel(writer, sheet_name='Students Trained', index=False)
    
    # Add monetary breakdown if available
    if not monetary_10yr.empty:
        monetary_10yr.to_excel(writer, sheet_name='Follow-on Funding 10yr')
    if not monetary_5yr.empty:
        monetary_5yr.to_excel(writer, sheet_name='Follow-on Funding 5yr')

print('\n✓ Executive summary exported to: IWRC_ROI_Analysis_Summary.xlsx')
print('\n✓ All visualizations saved as high-resolution PNG files:')
print('  • iwrc_investment_comparison.png')
print('  • followon_funding_count.png')
print('  • roi_comparison.png')
print('  • students_trained.png')
print('  • student_distribution_pie.png')

---
## Analysis Complete!

### Files Generated:
1. **IWRC_ROI_Analysis_Summary.xlsx** - Excel workbook with all summary tables
2. **High-resolution visualizations** (300 DPI PNG files):
   - Investment comparison chart
   - Follow-on funding count chart
   - ROI comparison chart
   - Students trained bar chart
   - Student distribution pie charts

### Next Steps:
- Use the visualizations in your one-pager fact sheet
- Reference the executive summary for key talking points
- Share the Excel workbook with stakeholders

---